In [5]:
%pip install numpy scikit-learn pandas nltk


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Question type of the parent question. 1 for conceptual questions, 2 for how-to questions, 3 for debug-corrective questions (Integer)

In [6]:
import pickle
import numpy as np
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import download

download('punkt') # Download the 'punkt' resource
download('punkt_tab')
download('wordnet') # Download the 'wordnet' resource
download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
data = []
# Load data from the pickle file
with open("data.pkl", "rb") as file:
    data = pickle.load(file)


df = pd.DataFrame(data)
df = df.drop_duplicates(subset='question_id')
df.to_csv('so_question_with_type.csv', index=False)

# Extract features (question titles) and labels (question types)
titles = df["question_title"] 
labels = df["question_type"]

df_questions = pd.DataFrame({
    'Question Title': titles,
    'Question Type': labels
})

print("Table of Question Titles and Types:")
print(df_questions.head(20))

question_type_counts = pd.DataFrame(df_questions['Question Type'].value_counts()).reset_index()
question_type_counts.columns = ['Question Type', 'Count']
question_type_counts.loc['Total'] = question_type_counts['Count'].sum()
print("\nDistribution of Question Types:")
print(question_type_counts)




Table of Question Titles and Types:
                                       Question Title  Question Type
0             How do I calculate someone's age in C#?              2
4    What are MVP and MVC and what is the difference?              1
7   How do you printf an unsigned long long int(th...              2
10                      What is Inversion of Control?              1
15  IllegalArgumentException or NullPointerExcepti...              3
20                       C# loop - break vs. continue              1
29                             Hidden Features of C#?              1
33  What's the best way to validate an XML file ag...              1
37  What is the difference between #include <filen...              1
38  Reading from text file until EOF repeats last ...              3
44  What issues should be considered when overridi...              1
45      Regular cast vs. static_cast vs. dynamic_cast              1
51   Why am I getting a NoClassDefFoundError in Java?              

In [9]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
X = vectorizer.fit_transform(titles)
y = np.array(labels)

# Split the data into train (80%), dev (10%), and test (10%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Train the SVM classifier
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train, y_train)

y_dev_pred = svm_model.predict(X_dev)
dev_accuracy = accuracy_score(y_dev, y_dev_pred)
print(f"Development Set Accuracy: {dev_accuracy:.2f}")

y_test_pred = svm_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Set Accuracy: {test_accuracy:.2f}")
print("Classification Report on Test Set:")
print(classification_report(y_test, y_test_pred, target_names=["how-to", "conceptual", "bug-fixing"]))

# with open("svm_question_classifier.pkl", "wb") as model_file:
#     pickle.dump(svm_model, model_file)


Development Set Accuracy: 0.57
Test Set Accuracy: 0.55
Classification Report on Test Set:
              precision    recall  f1-score   support

      how-to       0.91      0.32      0.48        31
  conceptual       0.51      0.87      0.64        38
  bug-fixing       0.45      0.28      0.34        18

    accuracy                           0.55        87
   macro avg       0.62      0.49      0.49        87
weighted avg       0.64      0.55      0.52        87



In [ ]:
# with open('question_classifier.pkl', 'rb') as model_file:
#     model = pickle.load(model_file)

# y_dev_pred = model.predict(X_dev.toarray())
# y_test_pred = model.predict(X_test.toarray())

# print("Dev Dataset Evaluation:")
# print(f"Accuracy: {accuracy_score(y_dev, y_dev_pred):.4f}")
# print(classification_report(y_dev, y_dev_pred))

# print("\nTest Dataset Evaluation:")
# print(f"Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
# print(classification_report(y_test, y_test_pred))